<a href="https://colab.research.google.com/github/FerrazThales/demand_forecasting_prophet/blob/main/Demand_Forecasting_with_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><b>Encontre eu e meus projetos nas redes sociais!</b></h1>
<table>
  <tr>
  <td><a href="https://thalesferraz.medium.com/">
  <img src="https://github.com/FerrazThales/FerrazThales/blob/main/logo_gif.gif?raw=true" width="800" title="Olá, Meu nome é Thales e sou cientista de Dados!"/>
  </a>
  </td>
  <td><a href="https://github.com/FerrazThales">
  <img hspace=30 vspace=110 src="https://image.flaticon.com/icons/png/512/1051/1051326.png" width="60%" title="Entre em meu Github e veja mais projetos!" /> 
  </a>
  </td>
  <td>
  <a href="">
  <img vspace=110 src="https://download.logo.wine/logo/Medium_(website)/Medium_(website)-Logo.wine.png" width="800" title="Veja este meu projeto no Medium!"/>
  </a>
  </td>
  <td><a href="https://www.linkedin.com/in/thalesdefreitasferraz/"><img vspace=150 src="https://image.flaticon.com/icons/png/512/889/889122.png" width="40%" title="Vamos trocar uma idéia sobre Data Science no LinkedIn?" />
  </a>
  </td>
  </tr>
</table>

# PREVISÃO DE DEMANDA

Como falamos antes, nossos templates ficarão cada vez mais simples!!

O objetivo desse projeto é prever a demanda por vinhos em lojas especializadas em vinho. Para esse projeto, temos 2 datasets, um sobre o catálogo de vinhos da loja e outro sobre as vendas.

* [Dados sobre os produtos](https://raw.githubusercontent.com/rafaelnduarte/sigmoidal_data/master/products.csv)
* [Dados de Vendas](https://raw.githubusercontent.com/rafaelnduarte/sigmoidal_data/master/sales-clean.csv)

Nesse projeto, é VITAL que haja MUITA ATENÇÃO sobre os detalhes mencionados que permeiam as análise de séries temporais. É um projeto que demanda muita atenção e cuidado, e uma documentação feita minuciosamente.



# O que você vai aprender nesta análise:

* x.
* x.
* x.
* x.

In [12]:
#importando as bibliotecas que serão utilizadas no projeto
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fbprophet import Prophet

# Texto Sobre previsão de demanda e vinhos

texto texto texto

imagem

texto

# Obtenção e ajuste dos dados

texto e 

### Dicionário das Variáveis:

> Dados Sobre os Produtos

* `item_id`: código do vinho na adega.
* `name`: nome do vinho.
* `producer`: nome do produtor do vinho.
* `country`: país de origem do vinho.
* `region`: região de produção do vinho.
* `vintage`: ano da safra.
* `kind`: tipo do vinho.

 * *Sparkling* : Espumante
 * *Rose Sparkling* : Espumante Rose
 * *White* : Branco
 * *Rosé* : Rosé
 * *Red* : Tinto
* `price_brl`: preço do vinho em real na época em que os dados foram levantados.
* `price_usd`: preço do vinho em dólar na época em que os dados foram levantados.

> Dados Sobre as Vendas

* `date`: data em que a venda foi realizada.
* `store`: xxx.
* `item`: código do vinho na adega.
* `sales`: quantidade de vinhos vendidos.


In [3]:
#endereco dos dados sobre produos
endereco_produtos = "https://raw.githubusercontent.com/rafaelnduarte/sigmoidal_data/master/products.csv"

#guardando os dados sobre produtos
produtos = pd.read_csv(endereco_produtos,index_col='item_id')

#mostrando as 5 primeiras entrada dos dados de produtos
produtos.head()

,name,producer,country,region,vintage,kind,price_brl,price_usd
item_id,,,,,,,,
1,Philipponnat Royale Reserve Brut,Philipponat,France,Champagne,NV,sparkling,339.60,58.75
2,Philipponnat Royale Reserve Rosé,Philipponat,France,Champagne,NV,rose sparkling,489.00,84.60
3,Philipponnat Cuvée 1522 Grand Cru Extra Brut,Philipponat,France,Champagne,2009,sparkling,789.00,136.51
4,Philipponnat Cuvée 1522 1er Cru Rosé,Philipponat,France,Champagne,2008,rose sparkling,899.40,155.61
5,Philipponnat Clos Des Goisses Brut,Philipponat,France,Champagne,2009,sparkling,"1,548.00",267.82


In [16]:
#endereco dos dados sobre vendas
endereco_vendas = "https://raw.githubusercontent.com/rafaelnduarte/sigmoidal_data/master/sales-clean.csv"

#guardando os dados sobre vendas
vendas = pd.read_csv(endereco_vendas)

#mostrando as 5 primeiras entrada dos dados de vendas
vendas.head()

,date,store,item,sales
0,2018-01-01,1,1,13
1,2018-01-02,1,1,11
2,2018-01-03,1,1,14
3,2018-01-04,1,1,13
4,2018-01-05,1,1,10


In [17]:
#vendo as 5 últimas entradas da base de dados sobre vendas
vendas.tail()

,date,store,item,sales
720066,12/27/2020,3,219,71
720067,12/28/2020,3,219,28
720068,12/29/2020,3,219,75
720069,12/30/2020,3,219,135
720070,12/31/2020,3,219,91


In [20]:
vendas.dtypes

date     object
store     int64
item      int64
sales     int64
dtype: object

In [14]:
vendas.store.unique()

array([1, 2, 3])

texto sobre a junção das duas bases de dados

In [ ]:
#códigos para juntar as duas bases de dados

# Análise Exploratória dos Dados

texto texto